# Рекомендация тарифов

# Описание задачи

Оператор мобильной связи собирается создать систему, способную проанализировав поведение клиентов предложить им новый тариф: "Смарт" или "Ультра". В распоряжении имеются данные о поведении клиентов, которые уже перешли на эти тарифы. Необходимо построить модель машинного обучения, которая выберет подходящий для клиента тариф. 

Построить модель с максимально большим значением accuracy, не менее 0.75.

# План работы

1. Обзор данных
    1. Чтение файлов с данными
2. Разбиение данных на выборки
3. Исследование моделей машинного обучения
    1. Модель "решающее дерево"
    2. Модель "случайный лес"
    3. Вывод
4. Проверка моделей на тестовой выборке
    1. Модель "решающее дерево"
    2. Модель "случайный лес"
    3. Вывод
5. Проверка моделей на адекватность
6. Вывод

# Описание данных

* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Обзор данных

### Чтение файлов с данными

In [2]:
#Импорт библиотеки pandas и функции деления на выборки
import pandas as pd
#Импорт функции деления на выборки
from sklearn.model_selection import train_test_split
#Импорт модели случайного леса
from sklearn.ensemble import RandomForestClassifier
#Импорт  модели решающего дерева
from sklearn.tree import DecisionTreeClassifier
#Импорт функции оценки точности
from sklearn.metrics import accuracy_score
#Импорт модели DummyClassifier
from sklearn.dummy import DummyClassifier

In [3]:
#Чтение таблицы
data = pd.read_csv('datasets/users_behavior.csv')

In [4]:
#Просмотр таблицы
data

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [5]:
#Проверка таблицы на пропуски
data.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

Вывод:

Названия столбцов указаны в едином регистре, пропуски отсутствуют.

## Разбиение данных на выборки

In [6]:
#Деление таблицы на обучающую и тестовую выборки в соотношении 60% на 40%
features_train, features_test = train_test_split(data, test_size=0.4, random_state=12345)

In [7]:
#Деление тестовой выборки на валидационную и тестовую в равных пропорциях
features_valid, features_test = train_test_split(features_test, test_size=0.5, random_state=12345)

In [8]:
#Размер обучающей выборки
features_train.shape

(1928, 5)

In [9]:
#Размер валидационной выборки
features_valid.shape

(643, 5)

In [10]:
#Размер тестовой выборки
features_test.shape

(643, 5)

Вывод:

Таблица поделена на три выборки - обучающую, валидационную и тестовую в соотношении 60%, 20%, 20%.

## Исследование моделей машинного обучения

Выполним подбор гиперпараметров для разных моделей

Гиперпараметры - max_depth (модель "решающее дерево"), n_estimators (модель "случайный лес")

### Модель "решающее дерево"

In [11]:
#Деление признаков обучающей выборки на целевой и остальные
target_train = features_train['is_ultra']
features_train = features_train.drop('is_ultra', axis=1)

In [12]:
#Деление признаков валидационной выборки на целевой и остальные
target_valid = features_valid['is_ultra']
features_valid = features_valid.drop('is_ultra', axis=1)

In [13]:
#Начальные значения переменных 
#лучшей модели
best_model = None
#лучшего результата accuracy
best_result = 0
#глубины дерева лучшей модели
best_depth = 0

In [14]:
#Определение лучшей модели, ее accuracy и глубины ее дерева
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth

In [15]:
#Вывод результатов определения accuracy и глубины дерева лучшей модели.
print("Accuracy лучшей модели:", best_result)
print("Глубина дерева лучшей модели:", best_depth)

Accuracy лучшей модели: 0.7853810264385692
Глубина дерева лучшей модели: 3


### Модель "случайный лес"

In [16]:
#Начальные значения переменных 
#лучшей модели
best_model = None
#лучшего результата accuracy
best_result = 0
#глубины дерева лучшей модели
best_depth = 0
#количества деревьев лучшей модели
best_est = 0

In [17]:
#Определение лучшей модели, ее accuracy, количества ее деревьев и их глубины
for est in range(1, 11):
    for depth in range(1, 6):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        result = accuracy_score(target_valid, predictions)
        if result > best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth
            
print("Accuracy наилучшей модели:", best_result)
print("Количество деревьев наилучшей модели:", best_est)
print("Глубина деревьев наилучшей модели:", best_depth)


Accuracy наилучшей модели: 0.7931570762052877
Количество деревьев наилучшей модели: 8
Глубина деревьев наилучшей модели: 5


### Вывод

Исследованы две модели - модель "решающее дерево" и модель "случайный лес". В модели "решающее дерево" была определена глубина дерева наилучшей модели равная 3. В модели "случайный лес" было определено количество деревьев и их глубина для наилучшей модели равные 8 и 5.

## Проверка моделей на тестовой выборке

### Модель "решающее дерево"

In [18]:
#Деление признаков тестовой выборки на целевой и остальные
target_test = features_test['is_ultra']
features_test = features_test.drop('is_ultra', axis=1)

In [19]:
#Сохранение модели решающего дерева в переменной model
model = DecisionTreeClassifier(random_state=12345, max_depth=3)

In [20]:
#Обучение модели на обучающей выборке
model.fit(features_train, target_train)

DecisionTreeClassifier(max_depth=3, random_state=12345)

In [21]:
#Предсказание обученной модели на тестовой выборке значений целевого признака
predictions = model.predict(features_test)

In [22]:
#Оценка точночти результатов предсказания
result = accuracy_score(target_test, predictions)

In [23]:
#Вывод полученной точности
print("Accuracy лучшей модели на тестовой выборке:", result)

Accuracy лучшей модели на тестовой выборке: 0.7791601866251944


Вывод: на тестовой выборке была проверена лучшая модель с глубиной дерева равной 3. Accuracy лучшей модели на тестовой выборке получилось равным 0.7791601866251944

### Модель "случайный лес"

In [24]:
#Сохранение модели случайного леса в переменной model
model = RandomForestClassifier(random_state=12345, n_estimators=8, max_depth=5)

In [25]:
#Обучение модели на обучающей выборке
model.fit(features_train, target_train)

RandomForestClassifier(max_depth=5, n_estimators=8, random_state=12345)

In [26]:
#Предсказание моделью значений целевого признака на тестовой выборке
predictions = model.predict(features_test)

In [27]:
#Оценка точности предсказания модели
result = accuracy_score(target_test, predictions)

In [28]:
#Вывод полученной точности
print("Accuracy лучшей модели на тестовой выборке:", result)

Accuracy лучшей модели на тестовой выборке: 0.7962674961119751


Вывод: на тестовой выборке была проверена лучшая модель с количеством деревьев равным 8 и глубиной дерева равной 3. Accuracy лучшей модели на тестовой выборке: 0.7962674961119751

### Вывод

На тестовой выборке были проверены две лучшие модели со значениями гиперпараметров найденными в предыдущем разделе 3. 
Accuracy лучшей модели "решающее дерево" на тестовой выборке оказалось равным 0.7791601866251944.  Accuracy лучшей модели "случайный лес" на тестовой выборке оказалось равным 0.7962674961119751. Оба значения не меньше 0.75, что соответствует требованиям.

## Проверка моделей на адекватность

Проверка модели на адекватность заключается в сравнении с константной моделью или моделью DummyClassifier.
Константная модель всегда предсказывает одно и то же значение. Чтобы предсказать значения целевого признака в тестовой выборке и свести ошибки к минимуму, будем предсказывать моду набора значений целевого признака обучающей выборки, которая, по сути, даст нам значение бОльшего класса.

In [29]:
#Сохранение модели DummyClassifier в переменной model
model = DummyClassifier()

In [30]:
#Обучение модели на обучающей выборке
model.fit(features_train, target_train)

DummyClassifier()

In [39]:
#Предсказание константной модели для валидационной выборки
predictions_valid = pd.Series(int(target_valid.mode().iloc[0]), index=target_valid.index)

In [40]:
#Результат предсказания, первые 5 строк
predictions_valid.head()

1386    0
3124    0
1956    0
2286    0
3077    0
dtype: int64

In [41]:
#Определение значения accuracy на валидационной выборке
accuracy_valid = accuracy_score(target_valid, predictions_valid)

In [42]:
#Предсказание константной модели для тестовой выборки
predictions_test = pd.Series(int(target_test.mode().iloc[0]), index=target_test.index)

In [43]:
#Определение значения accuracy на тестовой выборке
accuracy_test = accuracy_score(target_test, predictions_test)

In [44]:
#Вывод значений accuracy для обучающей и тестовой выборки
print ("Значение accuracy на валидационной выборке", accuracy_valid)
print("Значение accuracy на тестовой выборке", accuracy_test)

Значение accuracy на валидационной выборке 0.7060653188180405
Значение accuracy на тестовой выборке 0.6842923794712286


In [45]:
#Количество значений 0 у целевого признака обучающей выборки
target_train[target_train==0].count()

1335

In [46]:
#Общее количество значений целевого признака в обучающей выборке
target_train.shape

(1928,)

Модой набора значений целевого признака обучающей выборки оказалось значение 0. Соответственно, именно это значение и было выбрано для предсказания константной моделью для каждого объекта. Количество значений 0 в обучающей выборке у целевого признака равно 1335. Всего значений у целевого признака 1928 (0 и 1). Нетрудно подсчитать, что доля 0 составляет 69%, что соответствует значению accuracy 0,69.
    
У модели "решающее дерево" точность на тестовой выборке получилась равной 0.7791601866251944. Это значительно больше значения 0.6842923794712286. Модель "решающее дерево" адекватна.
    
У модели "случайный лес" точность на тестовой выборке получилась равной 0.7962674961119751. Это значительно больше значения 0.6842923794712286. Модель "случайный лес" адекватна.

## Вывод

Наилучшей моделью оказалась модель "случайный лес" с точностью равной 0.7962674961119751.